# Factory Functional Test

In [1]:
from pathfinder import Pathfinder

# abs_paths = ['']
# relative_paths = Pathfinder.relative_paths_from_abs(abs_paths)

relative_paths = [['..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
                  ['..', '..', 'codes']]

Pathfinder.append_relative_paths(relative_paths)

In [2]:
import pandas as pd

#https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

In [3]:
from bridges.ftdi.adapters.micropython import machine
from bridges.ftdi.controllers.i2c import I2cController
from sigma.sigma_dsp.adau import ADAU1401
from utilities.adapters import peripherals

## Debug mode?

In [4]:
cls = ADAU1401

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## DSP processor

In [5]:
with_hardware_device = False  # set True if hardware is connected.

if with_hardware_device:
    
    ctrl = I2cController()
    _machine = ctrl.get_gpio()

    _i2c = ctrl.I2C()

    _pin_reset = _machine.Pin('ADBUS4', mode = machine.Pin.OUT)
    _pin_reset.high()

else:
    _i2c = _pin_reset = None  # using None for testing without actual hardware device.

bus = peripherals.I2C(_i2c)


****** Virtual device. Data may not be real ! ******



In [6]:
# bus = fx2lp.I2C(as_400KHz = False)

In [7]:
dsp = ADAU1401(bus, pin_reset = _pin_reset)

## Factory

In [8]:
import os
from sigma.factory import Factory 

project_xml_file_url = os.sep.join(['..', '..', 'SigmaStudio projects', 'projects', 'demo', 'demo.xml'])
class_files_root_url = os.sep.join(['..', '..', 'codes', 'sigma', 'sigma_studio', 'toolbox', 'cells']) 

factory = Factory(project_xml_file_url = project_xml_file_url,
                  class_files_root_url = class_files_root_url,
                  dsp = dsp
                 )

## IC

In [9]:
# read and wrap project XML file into a "IC" object.

ic = factory.get_ic()
ic.df.tail(20)

param_full_name  \
algorithm_name  cell_name            param_name                                
SWGain1940DBAlg Volume_Control_in1   step               SWGain1940DBAlg4step   
                                     target           SWGain1940DBAlg4target   
                Volume_Control_out01 step               SWGain1940DBAlg2step   
                                     target           SWGain1940DBAlg2target   
                Volume_Control_out23 step               SWGain1940DBAlg3step   
                                     target           SWGain1940DBAlg3target   
SingleCtrlMixer Merger0                                 SingleCtrlMixer19401   
                Merger1                                 SingleCtrlMixer19402   
                Merger2                                 SingleCtrlMixer19403   
                Merger3                                 SingleCtrlMixer19404   
StMuxSwSlew     Source_Switch_0      coeffname         StMuxSwSlew1coeffname   
                Source_Switch_1      coeffname         StMuxSwSlew2coeffname   
WhiteNAlgNew    White_Noise          enable              WhiteNAlgNew1enable   
                                     mxx                    WhiteNAlgNew1mxx   
sin_lookupAlg   Tone1                increment   sin_lookupAlg19401increment   
                                     ison             sin_lookupAlg19401ison   
                                     mask             sin_lookupAlg19401mask   
                Tone2                increment   sin_lookupAlg19402increment   
                                     ison             sin_lookupAlg19402ison   
                                     mask             sin_lookupAlg19402mask   

                                                  type          value  \
algorithm_name  cell_name            param_name                         
SWGain1940DBAlg Volume_Control_in1   step        float       0.000244   
                                     target      float       1.000000   
                Volume_Control_out01 step        float       0.000244   
                                     target      float       0.173780   
                Volume_Control_out23 step        float       0.000244   
                                     target      float       0.158489   
SingleCtrlMixer Merger0                          float       0.250000   
                Merger1                          float       0.250000   
                Merger2                          float       0.250000   
                Merger3                          float       0.250000   
StMuxSwSlew     Source_Switch_0      coeffname   float       0.000000   
                Source_Switch_1      coeffname     int       2.000000   
WhiteNAlgNew    White_Noise          enable      float       1.000000   
                                     mxx           int  345675.000000   
sin_lookupAlg   Tone1                increment   float       0.004583   
                                     ison        float       1.000000   
                                     mask          int     255.000000   
                Tone2                increment   float       0.004750   
                                     ison        float       1.000000   
                                     mask          int     255.000000   

                                                 address  n_bytes  
algorithm_name  cell_name            param_name                    
SWGain1940DBAlg Volume_Control_in1   step             13        4  
                                     target           12        4  
                Volume_Control_out01 step            131        4  
                                     target          130        4  
                Volume_Control_out23 step            133        4  
                                     target          132        4  
SingleCtrlMixer Merger0                               58        4  
                Merger1                               59        4  
             

## Cell

In [10]:
# get_cell_by_name: get a "Cell" object by name.

cell = factory.get_cell_by_name('Volume_Control_in1')
cell.df

param_full_name   type  \
cell_name          algorithm_name  param_name                                  
Volume_Control_in1 SWGain1940DBAlg step          SWGain1940DBAlg4step  float   
                                   target      SWGain1940DBAlg4target  float   

                                                  value  address  n_bytes  
cell_name          algorithm_name  param_name                              
Volume_Control_in1 SWGain1940DBAlg step        0.000244       13        4  
                                   target      1.000000       12        4

## Cells 

In [11]:
# get all the "Cell" objects of a SigmaStudio project. A "Cell" has methods (like set_frequency...) with which we can control Dsp behavior.

cells = factory.get_cells()
cells['Volume_Control_in1'].df

param_full_name   type  \
cell_name          algorithm_name  param_name                                  
Volume_Control_in1 SWGain1940DBAlg step          SWGain1940DBAlg4step  float   
                                   target      SWGain1940DBAlg4target  float   

                                                  value  address  n_bytes  
cell_name          algorithm_name  param_name                              
Volume_Control_in1 SWGain1940DBAlg step        0.000244       13        4  
                                   target      1.000000       12        4

## Cell Manifest

In [12]:
# get_cells_manifest: generate a script to embody Cell objects.

for item in factory.get_cells_manifest()[-10:]:
    print(item)

param_eq_out3 = cells['Param EQ_out3']  # PEQ1Chan - Double Precision( 1 )
source_switch_0 = cells['Source_Switch_0']  # Stereo SW Slew( 3 )
source_switch_1 = cells['Source_Switch_1']  # Stereo SW Slew( 3 )
tone1 = cells['Tone1']  # Tone Synthesis (lookup/sine)( 1 )
tone2 = cells['Tone2']  # Tone Synthesis (lookup/sine)( 1 )
volume_control_in0 = cells['Volume_Control_in0']  # Gain (RC Slew)( 1 )
volume_control_in1 = cells['Volume_Control_in1']  # Gain (RC Slew)( 1 )
volume_control_out01 = cells['Volume_Control_out01']  # Gain (RC Slew)( 1 )
volume_control_out23 = cells['Volume_Control_out23']  # Gain (RC Slew)( 1 )
white_noise = cells['White_Noise']  # White Noise( 1 )


In [13]:
volume_control_out01 = cells['Volume_Control_out01']  # Gain (RC Slew)( 1 )
volume_control_out01.df

param_full_name  \
cell_name            algorithm_name  param_name                           
Volume_Control_out01 SWGain1940DBAlg step          SWGain1940DBAlg2step   
                                     target      SWGain1940DBAlg2target   

                                                  type     value  address  \
cell_name            algorithm_name  param_name                             
Volume_Control_out01 SWGain1940DBAlg step        float  0.000244      131   
                                     target      float  0.173780      130   

                                                 n_bytes  
cell_name            algorithm_name  param_name           
Volume_Control_out01 SWGain1940DBAlg step              4  
                                     target            4

## Cell's methods / properties description

In [14]:
# show a report of every Cell's methods signature in a SigmaStudio project.
    
#factory.show_methods()  ## un-comment this to show the long report.

In [15]:
# each cell can show its own functional description

volume_control_out01.show_methods()

{
    "cell_name": "Volume_Control_out01",
    "algorithm_name": "SWGain1940DBAlg",
    "methods": [
        "dB_to_gain(dB)",
        "gain_to_dB(gain)",
        "get_dB(param_name=None, **kwargs)",
        "get_gain(param_name=None, **kwargs)",
        "get_param(param_name='', algorithm_idx=0)",
        "get_parameters_values(**kwargs)",
        "read_parameter(param)",
        "set_dB(dB, param_name=None, **kwargs)",
        "set_gain(gain, param_name=None, **kwargs)",
        "set_param(data, param_name='', send_now=True, algorithm_idx=None)",
        "set_parameters_values(name_value_pairs, **kwargs)",
        "set_slew_rate(slew_rate, **kwargs)",
        "set_volume(dB, **kwargs)",
        "show_methods(print_out=True)",
        "write_parameter(param, send_now=True)"
    ],
    "properties": [
        "GAIN_PARAM",
        "PARAMETER_SHORT_NAMES",
        "SLEW_PARAM",
        "algorithm_name",
        "description",
        "df",
        "name",
        "parameters",
        "

In [16]:
# each cell can show its parameters in use

volume_control_out01.df

param_full_name  \
cell_name            algorithm_name  param_name                           
Volume_Control_out01 SWGain1940DBAlg step          SWGain1940DBAlg2step   
                                     target      SWGain1940DBAlg2target   

                                                  type     value  address  \
cell_name            algorithm_name  param_name                             
Volume_Control_out01 SWGain1940DBAlg step        float  0.000244      131   
                                     target      float  0.173780      130   

                                                 n_bytes  
cell_name            algorithm_name  param_name           
Volume_Control_out01 SWGain1940DBAlg step              4  
                                     target            4

In [17]:
# each cell can show its parameters in use

volume_control_out01.PARAMETER_SHORT_NAMES

['step', 'target']

In [18]:
# "properties" are the initial properties of the Algorithm, not every one of them are parameters.

volume_control_out01.properties

{'Gain`max': 0, 'Gain`min': -80, 'Gain`res': 100, 'Gain': -15.2}

In [19]:
# algorithm_name

volume_control_out01.algorithm_name

'SWGain1940DBAlg'

In [20]:
# description

volume_control_out01.description

'Gain (RC Slew)( 1 )'